<a href="https://colab.research.google.com/github/chad-fisher/SpatialDataScienceSandbox/blob/main/Geoprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>